# 🤖 AgentPro Quick Start Guide

This notebook will walk you through how to set up and use [AgentPro](https://github.com/traversaal-ai/AgentPro) — a production-ready open-source agent framework built by [Traversaal.ai](https://traversaal.ai) for building powerful, modular, and multi-functional AI agents.

### What is AgentPro?
AgentPro lets you build intelligent agents that can:
- Use language models (like OpenAI’s GPT) as reasoning engines
- Solve real-world tasks such as research, automation, and knowledge retrieval
- Scale up with custom tools, memory, and orchestration features

Whether you're a developer, researcher, or AI enthusiast — this guide will help you:
- Build and integrate your own tools with AgentPro


## Step 1: Clone AgentPro and Install Dependencies

To get started with **AgentPro**, begin by cloning the official GitHub repository and installing its dependencies.

In [1]:
!git clone https://github.com/traversaal-ai/AgentPro.git
%cd AgentPro
!pip install -r requirements.txt

Cloning into 'AgentPro'...
remote: Enumerating objects: 586, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 586 (delta 200), reused 117 (delta 98), pack-reused 304 (from 2)
Receiving objects: 100% (586/586), 224.79 KiB | 7.49 MiB/s, done.
Resolving deltas: 100% (333/333), done.
/content/AgentPro
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.7 MB/s eta 0:00:00


## Step 2: Set Your API Key

To use OpenAI models with AgentPro, you’ll need an API key from OpenAI. Follow these steps:

1. Go to the [OpenAI API platform](https://platform.openai.com/)
2. Log in or create an account
3. Click "Create new secret key"
4. Copy the generated key and paste it into the notebook like this:

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "<openai_api_key>"

## Step 3: Create a Custom Tool
AgentPro is designed to be extensible — you can easily define your own tools for domain-specific tasks.

Below is an example of a **custom tool** that queries the Hugging Face Hub and returns the **most downloaded model** for a given task:

In [3]:
from huggingface_hub import list_models

# Define the task you're interested in
task_name = "text-classification"

# Get the most downloaded model for the specified task
models = list_models(filter=task_name, sort="downloads", direction=-1)
top_model = next(iter(models))

# Print the model ID
print(top_model.id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


distilbert/distilbert-base-uncased-finetuned-sst-2-english


## Step 4: Define your tool using AgentPro Tool class

In [4]:
# Imports
import os
from agentpro import AgentPro
from agentpro.tools import Tool
from huggingface_hub import list_models
from typing import Any

class MostModelTool(Tool):
    name: str = "Most Downloaded Model Finder" # Human-readable tool name
    description: str = "Finds the most downloaded model for a given task on Hugging Face." # Brief explanation of tool for agent
    action_type: str = "find_top_model" # Use lowercase letters with underscores for agent; avoid spaces, digits and special characters
    input_format: str = "Task name as a string. Example: 'text-classification'" # Expected input dtype with example

    def run(self, input: Any) -> str:
        task_name = input.strip()
        models = list_models(filter=task_name, sort="downloads", direction=-1)
        top_model = next(models)
        return top_model.id

## Step 5: Pass tool to AgentPro

In [5]:
# Instantiate your tools and AgentPro
tools = [MostModelTool()]

myagent = AgentPro(model=os.getenv("OPENAI_API_KEY", None), tools=tools)

# Run
query = "Can you give me the name of the model that has the most downloads in the 'text-classification' task on the Hugging Face Hub?"
response = myagent.run(query)
# Can you give me the name of the model that has the most downloads in the 'text-classification' task on the Hugging Face Hub?
# Find the most popular model used for 'object-detection' on Hugging Face.

print(f"\nFinal Answer: {response.final_answer}")

================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: find_top_model

Tool: Most Downloaded Model Finder
Description: Finds the most downloaded model for a given task on Hugging Face.
Action Type: find_top_model
Input Format: Task name as a string. Example: 'text-classification'


For each iteration, follow these steps:
1. Thought: Think about what needs to be done.
2. Action: Decide on an appropriate Action if needed.
3. Observation: Observe the result after the Action.
4. PAUSE: Reflect on what to do next.
5. Repeat Thought/Action/Observation/PAUSE as needed until you find the final answer.

Format:
Thought: Your reasoning
Action: {"action_type": "<action_type>", "input": <input_data>}

Format after action 